In [1]:
%cd G:\내 드라이브\data\health

G:\내 드라이브\data\health


In [2]:
import pandas as pd
import pickle
import numpy as np
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, KFold

In [3]:
health = pd.read_csv('health.txt', index_col=0)

C:\Users\Jeongwon\anaconda3\envs\ml\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [4]:
data = health.drop(columns=['YEAR_t', 'trainset']).dropna()

In [5]:
data = data[data.age_MISS == 0]
data = data.drop(columns='age_MISS')

In [6]:
data.index = np.arange(0,138309)

In [7]:
data.to_pickle('health.pkl')

In [8]:
with open('health.pkl', 'rb') as f:
    df = pickle.load(f)

In [9]:
df.ClaimsTruncated = df.ClaimsTruncated.astype('int').astype('category')

In [10]:
age = df.apply(lambda x: 0 if x.loc['age_05':'age_65'].sum()==1 else 1, axis=1)

In [11]:
df = df.drop(columns=['age_05', 'age_15', 'age_25', 'age_35'
                 , 'age_45', 'age_55', 'age_65', 'age_75'
                 , 'age_85'])

In [12]:
df.DaysInHospital = df.DaysInHospital.apply(lambda x: 0 if x == 0 else 1)

In [13]:
categorical = ['ClaimsTruncated', 'DaysInHospital', 'sexMALE', 'sexFEMALE',
               'sexMISS', 'labNull', 'drugNull']

In [14]:
df_categorical = df[categorical]
df_numerical = df.drop(columns = categorical)

In [15]:
hot_categorical = pd.concat([df_categorical[x].astype('int').astype('category') for x in df_categorical], axis=1)
hot_categorical.columns = df_categorical.columns

In [16]:
one_hot_numeric = pd.concat([pd.cut(df_numerical[name], 5) for name in df_numerical], axis=1)
one_hot_numeric.columns = df_numerical.columns
one_hot_numeric = pd.concat([pd.get_dummies(one_hot_numeric[name], prefix=one_hot_numeric[name].name)
                                     for name in one_hot_numeric], axis=1)

In [17]:
one_hot = pd.concat([hot_categorical.iloc[:, 2:], one_hot_numeric, hot_categorical.iloc[:, 0:2]], axis=1)
one_hot['age'] = age

C:\Users\Jeongwon\anaconda3\envs\ml\lib\site-packages\ipykernel_launcher.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  


In [25]:
one_hot.rename(columns={'DaysInHospital': 'label'}, inplace=True)
one_hot.index= range(138309)

In [26]:
# train, test = train_test_split(one_hot, test_size=0.2)
# train.index = range(len(train))
# test.index = range(len(test))
# train.to_pickle('./health_train.pkl')
# test.to_pickle('./health_test.pkl')
kfold = KFold(n_splits=5)
for i, (train, test) in enumerate(kfold.split(one_hot)):
    train = one_hot[train]
    test = one_hot[test]
    train.index = range(len(train))
    test.index = range(len(test))
    train.to_pickle('./health_train_%d.pkl' % (i+1))
    test.to_pickle('./health_test_%d.pkl' % (i+1))

KeyError: "None of [Int64Index([ 27662,  27663,  27664,  27665,  27666,  27667,  27668,  27669,\n             27670,  27671,\n            ...\n            138299, 138300, 138301, 138302, 138303, 138304, 138305, 138306,\n            138307, 138308],\n           dtype='int64', length=110647)] are in the [columns]"

In [19]:
one_hot.head()

,sexMALE,sexFEMALE,sexMISS,labNull,drugNull,"no_Claims_(0.958, 9.4]","no_Claims_(9.4, 17.8]","no_Claims_(17.8, 26.2]","no_Claims_(26.2, 34.6]","no_Claims_(34.6, 43.0]",...,"labCount_ave_(6.0, 8.0]","labCount_ave_(8.0, 10.0]","labcount_months_(-0.012, 2.4]","labcount_months_(2.4, 4.8]","labcount_months_(4.8, 7.2]","labcount_months_(7.2, 9.6]","labcount_months_(9.6, 12.0]",ClaimsTruncated,label,age
0,0,1,0,0,1,1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0
2,1,0,0,1,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,0,0
3,0,1,0,1,1,1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,0,1,0,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,1,1
